In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", None)

# Load and Clean CSV with API Data

## Function

In [3]:
def load_and_clean_csv(party, csv_path):
    '''
    Function loads CSV data from the Twitter API+Sentiment and returns a cleaned DF
    '''
    # Load CSV Dataset via Path
    df = pd.read_csv(csv_path, lineterminator='\n', low_memory=False)
    
    # Create "Party" Column and rename other columns
    df['party'] = party
    df = df.rename(columns={"tweet_created_at": "tweet_date", 
                             "public_metrics.retweet_count": "retweet_count",
                             "public_metrics.reply_count": "reply_count",
                             "public_metrics.like_count": "like_count",
                             "profile_created_at": "profile_creation_date",
                             "public_metrics.followers_count": "followers_count",
                             "public_metrics.following_count": "following_count",
                             "public_metrics.tweet_count": "user_tweet_count"
                            })
    
    # Including only columns that we want to use in the future
    df = df[['party', 
             'tweet_date',
             'author_id',
             'tweet_id',
             'text',
             'source',
             'retweet_count',
             'reply_count',
             'like_count',
             'profile_creation_date',
             'followers_count',
             'following_count',
             'user_tweet_count',
             'location',
             'sentiment'
            ]]
    
    # Clean dataset columns:
       # Change dtype
    df["tweet_date"] = df["tweet_date"].astype(str)
    df = df[df.tweet_date.str.match('(\d{4}-\d{2}-\d{2}.\d{2}:\d{2}:\d{2})')]
    df = df[(df.tweet_date.str.len() == 23) | (df.tweet_date.str.len() == 24)]
    df['tweet_date'] = df['tweet_date'].str.slice(0,19)
    df["tweet_date"] = pd.to_datetime(df["tweet_date"])
    df['profile_creation_date'] = df['profile_creation_date'].str.slice(0,19)
    df["profile_creation_date"] = pd.to_datetime(df["profile_creation_date"])
       # Drop duplicates
    df = df.drop_duplicates()
       # Transform sentiment to numeric type
    dict_to_numeric = {"negative": -2, "neutral": 1, "positive": 2}
    df["sentiment"].replace(dict_to_numeric, inplace=True)

    return df

## Load CSV Data

In [4]:
df_cdu = load_and_clean_csv("CDU", "/Users/finnzurmuehlen/Downloads/2021_cdu_with_sentiment.csv")

In [5]:
df_linke = load_and_clean_csv("LINKE", '/Users/finnzurmuehlen/Downloads/2021_0723-0826_Tweets_Linken_sentiment.csv')

In [6]:
df_spd = load_and_clean_csv("SPD", '/Users/finnzurmuehlen/Downloads/2021_spd_with_sentiment.csv')

In [7]:
df_fdp = load_and_clean_csv("FDP", '/Users/finnzurmuehlen/Downloads/2021_gruene_with_sentiment.csv')

In [8]:
df_gruene = load_and_clean_csv("GRUENE", '/Users/finnzurmuehlen/Downloads/2021_fdp_with_sentiment.csv')

In [9]:
df_afd = load_and_clean_csv("AFD", '/Users/finnzurmuehlen/Downloads/2021_afd_with_sentiment.csv')

In [10]:
df_other = load_and_clean_csv("OTHER", '/Users/finnzurmuehlen/Downloads/2021_others_with_sentiment.csv')

# Concat. DataFrames

## Concat. Funtion

In [11]:
def concat_dfs(list_of_dfs):
    '''
    Function concatenates multiple dataframes into one DF
    '''
    df_all = pd.concat(list_of_dfs)
    df_all = df_all.reset_index(drop=True)
    return df_all

## Concat DFs

In [12]:
list_of_dfs = [df_cdu, df_spd, df_gruene, df_fdp, df_linke, df_afd, df_other]
df_all = concat_dfs(list_of_dfs)

In [29]:
df_all.to_csv("df_all.csv")

# Feature Engineering

## Engineering Function (Non-Sentiment Features)

In [14]:
def create_non_sentiment_features(df):
    
    # Create: "Len per tweet of each party"
    df["avg_len_of_tweet"] = df["text"].str.len()
    
    #Rename Columns
    df = df.rename(columns={"followers_count": "avg_followers_count",
                       "following_count": "avg_following_count", 
                       "user_tweet_count": "avg_user_tweet_count"
                      })
    # Change dtypes
    df = df.fillna(0)
    df["reply_count"] = df["reply_count"].astype(float)
    df["retweet_count"] = df["retweet_count"].astype(float)
    df["like_count"] = df["like_count"].astype(float)
    df["avg_len_of_tweet"] = df["avg_len_of_tweet"].astype(float)
    df["avg_followers_count"] = df["avg_followers_count"].astype(float)
    df["avg_following_count"] = df["avg_following_count"].astype(float)
    df["avg_user_tweet_count"] = df["avg_user_tweet_count"].astype(float)

    #Create temporary DF
    df_temp = df.groupby([pd.Grouper(key='tweet_date',freq='D'), 'party']).agg({
    "reply_count": "sum", 
    "retweet_count": "sum",
    "like_count": "sum",
    "avg_len_of_tweet": "mean",
    "avg_followers_count": "mean",
    "avg_following_count": "mean",
    "avg_user_tweet_count": "mean"
    }) 
    
    #Create: Followers Ratio
    df_temp["avg_ff_ratio"] = df_temp["avg_followers_count"] / df_temp["avg_following_count"]
    
    # Create: share of tweets that a party has in comparison to all tweets on a given day 
    df_temp_2 = df.groupby([pd.Grouper(key='tweet_date',freq='D'), 'party']).agg({
    "text": "count"}).groupby(level=0).apply(lambda x: x/x.sum())
    
    # Create: Share of tweets that come from a unique user for each party on a given day
    df_temp_3 = df.groupby([pd.Grouper(key='tweet_date',freq='D'), 'party']).agg({
    "author_id": "nunique",
    "text": "count"})
    df_temp_3["share_unique_users"] = df_temp_3["author_id"] / df_temp_3["text"]
    df_temp_3 = df_temp_3["share_unique_users"]
    
     # Join the different temporary DFs into a final DataFrame
    df_final = df_temp.join(df_temp_2).join(df_temp_3)
    df_final = df_final.rename(columns={'text': "share_of_tweets"})
    
    return df_final
    

In [15]:
df_non_sentiment = create_non_sentiment_features(df_all)
df_non_sentiment.tail(6)

reply_count  retweet_count  like_count  avg_len_of_tweet  \
tweet_date party                                                              
2021-08-26 CDU          7045.0        13120.0     98419.0        173.023969   
           FDP          7423.0         9063.0     63348.0        178.698807   
           GRUENE       3976.0         3199.0     29066.0        155.032074   
           LINKE         887.0          713.0      5282.0        190.801338   
           OTHER        1550.0         2166.0     11618.0        175.401146   
           SPD          8877.0        10807.0     89094.0        171.685269   

                   avg_followers_count  avg_following_count  \
tweet_date party                                              
2021-08-26 CDU             3929.033686           841.426474   
           FDP             4663.179284           690.310107   
           GRUENE          3785.480448           749.313049   
           LINKE            963.538462           684.646823   
           OTHER           1340.969436           638.417861   
           SPD             6223.901363           887.675427   

                   avg_user_tweet_count  avg_ff_ratio  share_of_tweets  \
tweet_date party                                                         
2021-08-26 CDU             22336.432628      4.669491         0.225380   
           FDP             20607.243817      6.755195         0.193819   
           GRUENE          17960.210457      5.051935         0.110768   
           LINKE           20400.862207      1.407351         0.036379   
           OTHER           14171.402579      2.100457         0.050955   
           SPD             25279.190893      7.011461         0.224991   

                   share_unique_users  
tweet_date party                       
2021-08-26 CDU               0.646945  
           FDP               0.638167  
           GRUENE            0.716828  
           LINKE             0.590635  
           OTHER             0.579752  
           SPD               0.617673

## Engineering Function (Sentiment Features)

In [16]:
import numpy as np

In [17]:
def create_sentiment_features(df):
    '''
    Generates the following features: "Weighted Sentiment", "Share of positive tweets", "Share of negative tweets".
    '''
    # Change dtype
    df = df.fillna(0)
    df["retweet_count"] = df["retweet_count"].astype(float)
    df["like_count"] = df["like_count"].astype(float)
    df["sentiment"] = df["sentiment"].astype(float)
    
    df = df[["tweet_date","party","retweet_count", "like_count", "sentiment"]]
    # Generate "Weighted Sentiment"
    df["like_count"] = df["like_count"]+10
    df["retweet_count"] = df["retweet_count"]+10
    df["weighted_sentiment"] = np.log10(df["like_count"]) * np.log10(df["retweet_count"]) * df["sentiment"]
    
    # Generate "Share of positive tweets"
    df["share_of_positive_tweets"] = df["sentiment"]    
    dict_only_positive = {-2: 0, 1: 0, 2: 1}
    df["share_of_positive_tweets"].replace(dict_only_positive, inplace=True)
    
    # Generate "Share of negative tweets"
    df["share_of_negative_tweets"] = df["sentiment"]
    dict_only_negative = {-2: 1, 1: 0, 2: 0}
    df["share_of_negative_tweets"].replace(dict_only_negative, inplace=True)
    
    df["share_of_positive_tweets2"] = df["share_of_positive_tweets"]
    df["share_of_negative_tweets2"] = df["share_of_negative_tweets"]
    df = df.groupby([pd.Grouper(key='tweet_date',freq='D'), "party"]).agg({
        "weighted_sentiment": "mean",
        "share_of_positive_tweets": "sum",
        "share_of_positive_tweets2": "count",
        "share_of_negative_tweets": "sum",
        "share_of_negative_tweets2": "count",})
    df["share_of_positive_tweets"] = df["share_of_positive_tweets"] / df["share_of_positive_tweets2"]
    df["share_of_negative_tweets"] = df["share_of_negative_tweets"] / df["share_of_negative_tweets2"]
    df = df.drop(columns=["share_of_positive_tweets2", "share_of_negative_tweets2"])
    
    return df

In [18]:
df_sentiment = create_sentiment_features(df_all)

## Join Function (Combine Sentiment & Non-Sentiment Features)

In [19]:
def join_features(df1, df2):
    df_joined = df1.join(df2)
    return df_joined

In [20]:
df_joined = join_features(df_non_sentiment, df_sentiment)
df_joined.head()

reply_count  retweet_count  like_count  avg_len_of_tweet  \
tweet_date party                                                              
2021-07-23 AFD          5414.0         7430.0     50589.0        183.483988   
           CDU          5430.0         8987.0     53965.0        191.227455   
           FDP          6288.0         7621.0     60041.0        182.306444   
           GRUENE       2551.0         1854.0     15224.0        171.541570   
           LINKE         401.0          268.0      1625.0        181.312292   

                   avg_followers_count  avg_following_count  \
tweet_date party                                              
2021-07-23 AFD             9897.783065           844.707798   
           CDU             5095.104329           921.350581   
           FDP             3682.779195           773.788140   
           GRUENE          4400.461124           791.273287   
           LINKE           1976.892027           746.588040   

                   avg_user_tweet_count  avg_ff_ratio  share_of_tweets  \
tweet_date party                                                         
2021-07-23 AFD             30258.523792     11.717405         0.241723   
           CDU             26716.583105      5.530039         0.207085   
           FDP             24578.680470      4.759415         0.264028   
           GRUENE          26521.715935      5.561241         0.113623   
           LINKE           36722.935216      2.647902         0.026328   

                   share_unique_users  weighted_sentiment  \
tweet_date party                                            
2021-07-23 AFD               0.599421           -0.822698   
           CDU               0.673073           -0.466696   
           FDP               0.619016           -0.267626   
           GRUENE            0.675905           -0.385271   
           LINKE             0.616279           -0.050804   

                   share_of_positive_tweets  share_of_negative_tweets  
tweet_date party                                                       
2021-07-23 AFD                     0.028768                  0.587842  
           CDU                     0.035058                  0.480465  
           FDP                     0.060460                  0.439125  
           GRUENE                  0.061971                  0.476905  
           LINKE                   0.061462                  0.373754

# Final DataFrame

## Load Poll API Data

In [21]:
def load_poll_df(path):
    df = pd.read_csv(path)[["Date", "CDU/CSU", 'SPD', 'Grüne', 'FDP', "Linke", 'AfD', 'other']]
    return df

In [22]:
df_poll = load_poll_df('/Users/finnzurmuehlen/Downloads/polls_data_2021_v4.csv')

## Function: Create Final DF for RNN

In [23]:
def create_rnn_final_df(df_poll ,df_joined):
    '''
    Joines (how=outer) engineered features DF and poll DF for the German parties
    '''
    # Rename df_poll columns and change dtype to datetime
    df_poll = df_poll.rename(columns = {"Date": "tweet_date", 
                                        "CDU/CSU":"CDU",
                                        "Grüne": "GRUENE",
                                        "Linke": "LINKE",
                                        "AfD": "AFD",
                                        "other": "OTHER"
                                       })
    df_poll["tweet_date"] = pd.to_datetime(df_poll["tweet_date"])
    df_poll = df_poll.set_index("tweet_date")
    
    # Unstack the indexes in order to join on the tweet date and parties
    df_poll = pd.DataFrame(df_poll.T.unstack(level = 0))
    df_poll.index = df_poll.index.set_names(['tweet_date', 'party'])
    
    # Join both DFs together
    df_final = df_poll.join(df_joined, how = "outer")
    
    # Rename new column as "poll"
    df_final = df_final.rename(columns = {0: "poll"})
    
    return df_final

## Create Final Dataframe

In [26]:
df_final = create_rnn_final_df(df_poll, df_joined)
df_final.tail(7)

poll  reply_count  retweet_count  like_count  \
tweet_date party                                                  
2021-08-26 AFD     11.0       6354.0         7848.0     48207.0   
           CDU     23.0       7045.0        13120.0     98419.0   
           FDP     12.0       7423.0         9063.0     63348.0   
           GRUENE  18.0       3976.0         3199.0     29066.0   
           LINKE    7.0        887.0          713.0      5282.0   
           OTHER    6.0       1550.0         2166.0     11618.0   
           SPD     23.0       8877.0        10807.0     89094.0   

                   avg_len_of_tweet  avg_followers_count  avg_following_count  \
tweet_date party                                                                
2021-08-26 AFD           171.556241          2003.584015           761.860824   
           CDU           173.023969          3929.033686           841.426474   
           FDP           178.698807          4663.179284           690.310107   
           GRUENE        155.032074          3785.480448           749.313049   
           LINKE         190.801338           963.538462           684.646823   
           OTHER         175.401146          1340.969436           638.417861   
           SPD           171.685269          6223.901363           887.675427   

                   avg_user_tweet_count  avg_ff_ratio  share_of_tweets  \
tweet_date party                                                         
2021-08-26 AFD             18991.635859      2.629856         0.157708   
           CDU             22336.432628      4.669491         0.225380   
           FDP             20607.243817      6.755195         0.193819   
           GRUENE          17960.210457      5.051935         0.110768   
           LINKE           20400.862207      1.407351         0.036379   
           OTHER           14171.402579      2.100457         0.050955   
           SPD             25279.190893      7.011461         0.224991   

                   share_unique_users  weighted_sentiment  \
tweet_date party                                            
2021-08-26 AFD               0.623978           -0.830083   
           CDU               0.646945           -0.370421   
           FDP               0.638167           -0.149995   
           GRUENE            0.716828           -0.577197   
           LINKE             0.590635           -0.134164   
           OTHER             0.579752           -0.129097   
           SPD               0.617673           -0.255832   

                   share_of_positive_tweets  share_of_negative_tweets  
tweet_date party                                                       
2021-08-26 AFD                     0.047061                  0.596667  
           CDU                     0.042647                  0.463291  
           FDP                     0.062775                  0.406026  
           GRUENE                  0.056459                  0.527460  
           LINKE                   0.060201                  0.400669  
           OTHER                   0.087393                  0.409742  
           SPD                     0.073221                  0.443543

In [28]:
df_final.to_csv('data_final_20210826_v1.csv')